In [2]:
import xml.etree.ElementTree as ElementTree
import os, sys, csv
from urllib.request import urlopen
import pandas as pd 
import numpy as np
import requests
import xml.etree.ElementTree as ET

class XML2DataFrame:
    root = ''
    def __init__(self, xml_data):
        self.root = ET.XML(xml_data)


    def parse_root(self, root):
        return [self.parse_element(child) for child in iter(root)]

    def parse_element(self, element, parsed=None):
        if parsed is None:
            parsed = dict()
        for key in element.keys():
            parsed[key] = element.attrib.get(key)
        if element.text:
            parsed[element.tag] = element.text
        for child in list(element):
            self.parse_element(child, parsed)
        return parsed

    def process_data(self,root):
        structure_data = self.parse_root(root)
        return pd.DataFrame(structure_data)

In [77]:
def single_quote(s1):
    return "'%s'" % s1
    
user_agent_url = 'https://scsanctions.un.org/al-qaida/'
fetched_data = requests.get(user_agent_url).content


In [76]:
# user_agent_url = 'https://scsanctions.un.org/al-qaida/'
# url = user_agent_url
# fetched_data = requests.get(user_agent_url).content
# from bs4 import BeautifulSoup
# soup = BeautifulSoup(fetched_data)
# import re
# links = soup.findAll('a', attrs={'href': re.compile("xmlFullSanctionsList")})
# for link in links:
#     url = link.text


In [11]:
url = user_agent_url

import urllib
urllib.request.urlretrieve(url, 'temp.csv')
#from lxml import etree
#xml = etree.parse('temp.csv')


('temp.csv', <http.client.HTTPMessage at 0x1144de240>)

In [23]:
with open('temp.csv','r', encoding="utf8") as f:
    fetched_data = f.read()

In [24]:
root = ET.XML(fetched_data)
# name_tags = ['Id','WHOLENAME','FUNCTION']
# alt_name_tags = ['Id','WHOLENAME']

name_tags =  ['dataid', 'first_name', 'second_name', 'third_name', 'fourth_name', 'comments1']
alt_name_tags =  ['dataid', 'alias_name']


In [25]:
xml2df = XML2DataFrame(fetched_data)

In [81]:
for attribute in xml2df.root:
    for indiv in attribute.iter('INDIVIDUALS'):
        df_indv = xml2df.process_data(indiv)


In [82]:
for attribute in xml2df.root:
    for entity in attribute.iter('ENTITIES'):
        df_ent = xml2df.process_data(entity)


In [83]:
alt_df = df_indv[alt_name_tags]
alt_df.head()

,DATAID,ALIAS_NAME
0,6908576,Abu-Julaybib
1,6908757,Taha al-Khuwayt
2,6908443,'Umar al-Tayyar
3,6908440,Ibn al-Khattab
4,111900,Abu Ayub


In [79]:
for attribute in xml2df.root:
    for indiv in xml2df.root.iter('INDIVIDUAL'):
        indv = xml2df.process_data(indiv)

    


In [80]:
indv.head()

,ALIAS_NAME,COMMENTS1,DATAID,DATE,DATE_OF_BIRTH,DATE_OF_ISSUE,FIRST_NAME,ISSUING_COUNTRY,LISTED_ON,NAME_ORIGINAL_SCRIPT,...,NUMBER,QUALITY,REFERENCE_NUMBER,SECOND_NAME,TYPE_OF_DATE,TYPE_OF_DOCUMENT,TYPE_OF_DOCUMENT2,UN_LIST_TYPE,VALUE,VERSIONNUM
0,NaN,NaN,2959427,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,NaN,NaN,NaN,NaN,NaN,NaN,SAID JAN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,‘ABD AL-SALAM,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Al-Qaida,NaN,NaN


In [30]:
df_ent.head()

,ALIAS_NAME,CITY,COMMENTS1,COUNTRY,DATAID,FIRST_NAME,LISTED_ON,NAME_ORIGINAL_SCRIPT,NOTE,QUALITY,REFERENCE_NUMBER,STATE_PROVINCE,STREET,SUBMITTED_ON,UN_LIST_TYPE,VALUE,VERSIONNUM,ZIP_CODE
0,Yusuf al-'Uyayri Battalions of the Abdallah Az...,NaN,An armed group that has carried out joint atta...,NaN,6908409,ABDALLAH AZZAM BRIGADES (AAB),2014-09-23,NaN,"Operates in Lebanon, Syria and the Arabian Pen...",a.k.a.,QDe.144,NaN,NaN,NaN,Al-Qaida,UN List,1,NaN
1,Al Harakat Al Islamiyya,NaN,Associated with Jemaah Islamiyah (JI) (QDe.092...,Philippines,113445,ABU SAYYAF GROUP,2001-10-06,NaN,NaN,a.k.a.,QDe.001,NaN,NaN,NaN,Al-Qaida,2011-12-13,1,NaN
2,Ahya ul Turas,Jalabad,Associated with the Revival of Islamic Heritag...,Afghanistan,113226,AFGHAN SUPPORT COMMITTEE (ASC),2002-01-11,NaN,NaN,a.k.a.,QDe.069,NaN,"Cheprahar Hadda, Mia Omar Sabaqah School",NaN,Al-Qaida,2011-12-13,1,NaN
3,Citizens’ Association for Support and Preventi...,Zavidovici,Registered in Bosnia and Herzegovina as a citi...,Bosnia and Herzegovina,113231,AL FURQAN,2004-05-11,NaN,NaN,a.k.a.,QDe.107,NaN,Zlatnih Ljiljana Street,NaN,Al-Qaida,2009-03-24,1,NaN
4,Those Who Sign in Blood,NaN,Associated with the Organization of Al-Qaida i...,Mali,6908426,AL MOUAKAOUNE BIDDAM,2014-06-02,الموقعون بالدم,NaN,a.k.a.,QDe.139,NaN,NaN,NaN,Al-Qaida,UN List,1,NaN


In [39]:
name_tags = ['DATAID', 'FIRST_NAME', 'SECOND_NAME', 'THIRD_NAME', 'FOURTH_NAME', 'COMMENTS1']
alt_name_tags = ['DATAID', 'ALIAS_NAME']
addr_tags = ['DATAID', 'STREET', 'CITY', 'COUNTRY']

In [ ]:
for attribute in xml2df.root:
    for entity in attribute.iter('ENTITIES'):
        df_ent = xml2df.process_data(entity)


In [57]:
df_ent.columns

Index(['ALIAS_NAME', 'CITY', 'COMMENTS1', 'COUNTRY', 'DATAID', 'FIRST_NAME',
       'LISTED_ON', 'NAME_ORIGINAL_SCRIPT', 'NOTE', 'QUALITY',
       'REFERENCE_NUMBER', 'STATE_PROVINCE', 'STREET', 'SUBMITTED_ON',
       'UN_LIST_TYPE', 'VALUE', 'VERSIONNUM', 'ZIP_CODE'],
      dtype='object')

In [60]:
df_alias_name = df_ent[alt_name_tags]
df_alias_name.head()



,DATAID,ALIAS_NAME
0,6908409,Yusuf al-'Uyayri Battalions of the Abdallah Az...
1,113445,Al Harakat Al Islamiyya
2,113226,Ahya ul Turas
3,113231,Citizens’ Association for Support and Preventi...
4,6908426,Those Who Sign in Blood


In [64]:
df_addr = df_ent[addr_tags]
df_addr.head()

df_name = df_indv[name_tags]
df_name.head()


,DATAID,FIRST_NAME,SECOND_NAME,THIRD_NAME,FOURTH_NAME,COMMENTS1
0,6908576,IYAD,NAZMI,SALIH,KHALIL,Leader of Al-Nusrah Front for the People of th...
1,6908757,HAJJI,'ABD AL-NASIR,NaN,NaN,ISIL military leader in the Syrian Arab Republ...
2,6908443,'ABD AL-MALIK,MUHAMMAD,YUSUF,'UTHMAN 'ABD AL-SALAM,"Facilitator who provides financial, material, ..."
3,6908440,ASHRAF,MUHAMMAD,YUSUF,'UTHMAN 'ABD AL-SALAM,A member of Al-Qaida (QDe.004) as of 2012 and ...
4,111900,NASHWAN,ABD AL-RAZZAQ,ABD AL-BAQI,NaN,Joined Al-Qaida in 1996 and was at that time a...


In [65]:

                      

#df ['DATAID'] = df ['DATAID'].apply (lambda x : x.split("_")[0])
#df ['ID'] = df ['DATAID'].apply(lambda x: 'EU'.str(x))



    
    

,ALIAS_NAME,CITY,CITY_OF_BIRTH,CITY_OF_ISSUE,COMMENTS1,COUNTRY,COUNTRY_OF_BIRTH,COUNTRY_OF_ISSUE,DATAID,DATE,...,THIRD_NAME,TO_YEAR,TYPE_OF_DATE,TYPE_OF_DOCUMENT,TYPE_OF_DOCUMENT2,UN_LIST_TYPE,VALUE,VERSIONNUM,YEAR,ZIP_CODE
0,Abu-Julaybib,,,,Leader of Al-Nusrah Front for the People of th...,Syrian Arab Republic,,,6908576,,...,SALIH,,EXACT,Passport,,Al-Qaida,UN List,1,1974,
1,Taha al-Khuwayt,Tall ''''Afar,,,ISIL military leader in the Syrian Arab Republ...,Iraq,,,6908757,,...,,1969,BETWEEN,,,Al-Qaida,UN List,1,,
2,''''Umar al-Tayyar,,,,"Facilitator who provides financial, material, ...",,,Qatar,6908443,1989-07-13,...,YUSUF,,EXACT,National Identification Number,,Al-Qaida,UN List,1,,
3,Ibn al-Khattab,,,,A member of Al-Qaida (QDe.004) as of 2012 and ...,Iraq,,Qatar,6908440,,...,YUSUF,,EXACT,National Identification Number,,Al-Qaida,UN List,1,1984,
4,Abu Ayub,Mosul,,,Joined Al-Qaida in 1996 and was at that time a...,Iraq,,,111900,,...,ABD AL-BAQI,,EXACT,National Identification Number,Ration Card no,Al-Qaida,2015-12-10,1,1961,


In [73]:
import json
for index, row in df.iterrows():
    ID1 = single_quote('') 
    addr = {'street' : row['STREET'], 'city': row['CITY'], 'country' : row['COUNTRY']}
                                                                           
    query_value = "( " + single_quote(row['DATAID']) + ", " + ID1 + ", " +single_quote(json.dumps(addr))+ ", "+single_quote(row['UN_LIST_TYPE']) + ") "
    insert_command = "insert into [ref].[sanctions_addresses_new]  (Id, ID1,Address,source) values"
    query = '{} {}'.format(insert_command, query_value)
    print("Index : " )
    print(index)
    print(query)
    #stat,message = db_insert(query)

Index : 
0
insert into [ref].[sanctions_addresses_new]  (Id, ID1,Address,source) values ( '6908576', '', '{"country": "Syrian Arab Republic", "street": "", "city": ""}', 'Al-Qaida') 
Index : 
1
insert into [ref].[sanctions_addresses_new]  (Id, ID1,Address,source) values ( '6908757', '', '{"country": "Iraq", "street": "", "city": "Tall ''''Afar"}', 'Al-Qaida') 
Index : 
2
insert into [ref].[sanctions_addresses_new]  (Id, ID1,Address,source) values ( '6908443', '', '{"country": "", "street": "", "city": ""}', 'Al-Qaida') 
Index : 
3
insert into [ref].[sanctions_addresses_new]  (Id, ID1,Address,source) values ( '6908440', '', '{"country": "Iraq", "street": "", "city": ""}', 'Al-Qaida') 
Index : 
4
insert into [ref].[sanctions_addresses_new]  (Id, ID1,Address,source) values ( '111900', '', '{"country": "Iraq", "street": "", "city": "Mosul"}', 'Al-Qaida') 
Index : 
5
insert into [ref].[sanctions_addresses_new]  (Id, ID1,Address,source) values ( '111533', '', '{"country": "Egypt", "street": 

In [397]:
df.head()

,DATAID,ADDRESS
0,6908473_1,United Kingdom of Great Britain and Northern I...
1,6908416_0,None
2,6908470_0,Qatar
3,6908424_0,None
4,6908533_0,Libya
